In [31]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import os
import numpy as np
from os import listdir
from pathlib import Path
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import h5py
import pydicom as dicom
import matplotlib.pyplot as plt
from dicom_contour.contour import get_contour_file,get_roi_names, coord2pixels, cfile2pixels, plot2dcontour, slice_order, get_contour_dict, get_data,  create_image_mask_files
#from preprocessing.get_plan_from_dicom import Plan, plot_DVH, structure_range, img_cut, resample
#from preprocessing.get_plan_from_dicom import plot_3d_img, plan_unit_test
#from preprocessing.generate_data import get_plans_unit_test
from util import *
from config import *
#from preprocessing.generate_data import get_plans
from dicom_contour.contour import get_ct_name_dict
from dicom_contour.dose import build_dose_volume
import pickle
from PIL import Image, ImageDraw
from copy import deepcopy
import math
from pytorch_programs.data_loader import PredictionDataSet
import torch
from torch.utils import data
from pytorch_programs.unet_model import UNet
from pytorch_programs.trainer import Trainer
from pytorch_programs.inference import predict_unit_test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
## generate plans for all plans
data_folder = training_npy_path
training_dataset = PredictionDataSet(data_folder = data_folder)
training_dataloader = data.DataLoader(dataset=training_dataset,
                                      batch_size=2,
                                      shuffle=True)
x, y = next(iter(training_dataloader))
print(f'x = shape: {x.shape}; type: {x.dtype}')
print(f'x = min: {x.min()}; max: {x.max()}')
print(f'y = shape: {y.shape}; class: {y.unique()}; type: {y.dtype}')


x = shape: torch.Size([2, 32, 96, 128, 12]); type: torch.float32
x = min: 0.0; max: 1.0
y = shape: torch.Size([2, 32, 96, 128, 1]); class: tensor([ 1.6877,  1.6966,  1.7008,  ..., 27.4813, 27.5234, 27.6154]); type: torch.float32


In [12]:
model = UNet(n_channels=12, n_classes=1)

x = torch.randn(size=(1, 12, 32, 96, 128), dtype=torch.float32)
with torch.no_grad():
    out = model(x)

print(f'Out: {out.shape}')

Out: torch.Size([1, 1, 32, 96, 128])


In [13]:
from torchsummary import summary
summary = summary(model, (12, 32, 96,128), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1      [-1, 16, 32, 96, 128]           5,200
              ReLU-2      [-1, 16, 32, 96, 128]               0
            Conv3d-3      [-1, 16, 32, 96, 128]          12,112
              ReLU-4      [-1, 16, 32, 96, 128]               0
        DoubleConv-5      [-1, 44, 32, 96, 128]               0
         MaxPool3d-6       [-1, 44, 16, 48, 64]               0
            Conv3d-7       [-1, 16, 16, 48, 64]          19,024
              ReLU-8       [-1, 16, 16, 48, 64]               0
            Conv3d-9       [-1, 16, 16, 48, 64]          25,936
             ReLU-10       [-1, 16, 16, 48, 64]               0
       DoubleConv-11       [-1, 76, 16, 48, 64]               0
             Down-12       [-1, 76, 16, 48, 64]               0
        MaxPool3d-13        [-1, 76, 8, 24, 32]               0
           Conv3d-14        [-1, 16, 8,

In [5]:
data_folder = training_npy_path
training_dataset = PredictionDataSet(data_folder = data_folder)
dataloader_training = data.DataLoader(dataset=training_dataset,
                                      batch_size=2,
                                      shuffle=True)

dataloader_validation = data.DataLoader(dataset=training_dataset,
                                      batch_size=2,
                                      shuffle=True)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    torch.device('cpu')

# model
model = UNet(n_channels=12, n_classes=1).to(device)



In [7]:
# criterion
criterion = torch.nn.MSELoss()

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

# trainer
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  training_DataLoader=dataloader_training,
                  validation_DataLoader=dataloader_validation,
                  lr_scheduler=None,
                  epochs=100,
                  epoch=0,
                  notebook=True)

# start training
training_losses, validation_losses, lr_rates = trainer.run_trainer()

Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/10 [00:00<?, ?it/s]

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/10 [00:00<?, ?it/s]

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/10 [00:00<?, ?it/s]

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/10 [00:00<?, ?it/s]

Validation:   0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
model_name =  'test_model.pt'
torch.save(model.state_dict(), str(Path.cwd())+'/pytorch_programs'+'/'+model_name)

In [25]:
model_name =  'test_model.pt'
str(Path.cwd())+'/pytorch_programs'+'/'+model_name

'c:\\ddu\\radiotherapy-dose-prediction/pytorch_programs/test_model.pt'

In [32]:
predict_unit_test()

Data/npy_dataset/training/
data_dirs is ['batch_0.npy', 'batch_1.npy', 'batch_10.npy', 'batch_11.npy', 'batch_12.npy', 'batch_13.npy', 'batch_14.npy', 'batch_15.npy', 'batch_16.npy', 'batch_17.npy', 'batch_18.npy', 'batch_19.npy', 'batch_2.npy', 'batch_3.npy', 'batch_4.npy', 'batch_5.npy', 'batch_6.npy', 'batch_7.npy', 'batch_8.npy', 'batch_9.npy']


IndexError: list index out of range

In [ ]:
print(metrics_all.keys())
metrics_all['patient_4']